In [1]:
pip install mlflow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# install the latest release candidate
pip install --pre mlflow

# or install a specific rc version
pip install mlflow==2.14.0rc0

In [ ]:
!mlflow server --host 127.0.0.1 --port 5000 # Run this command in CMD prompt

In [1]:
import mlflow

# Define the MLflow tracking URI
uri = "http://127.0.0.1:5000"

# Set the MLflow tracking URI
mlflow.set_tracking_uri(uri)

print(f"MLflow tracking URI is set to {uri}")

MLflow tracking URI is set to http://127.0.0.1:5000


In [35]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

1.0


In [37]:
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Quickstart-3")

# Start an MLflow run
with mlflow.start_run():
    
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart-3",
    )

Registered model 'tracking-quickstart-3' already exists. Creating a new version of this model...
2024/12/10 06:39:56 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart-3, version 2


🏃 View run powerful-mouse-307 at: http://127.0.0.1:5000/#/experiments/241483747149809741/runs/c40ac7a372574a708a7bf5ca8f35b8c6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/241483747149809741


Created version '2' of model 'tracking-quickstart-3'.


In [29]:
# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)
print(loaded_model)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]

mlflow.pyfunc.loaded_model:
  artifact_path: iris_model
  flavor: mlflow.sklearn
  run_id: 09cf289bba1b43998cb1cc5049cfd803



,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1
